In [1]:
from webcrawler import Crawler, Smartphone, Smartwatch, Laptop, Screen, Tablet, Earphone,Product
import json

In [39]:
urls = [
    "https://www.thegioididong.com/dtdd/samsung-galaxy-s25-ultra",  # smartphone
    "https://www.thegioididong.com/dtdd/nokia-220-4g",  # phone
    "https://www.thegioididong.com/laptop/macbook-air-13-inch-m4-16gb-256gb-sac-70w",  # lapatop
    "https://www.thegioididong.com/tai-nghe/tai-nghe-tws-jbl-wave-beam-2?utm_flashsale=1",  # tws
    "https://www.thegioididong.com/tai-nghe/tai-nghe-co-day-gaming-hp-hyperx-cloud-stinger-core-ii",  # headphone
    "https://www.thegioididong.com/tai-nghe/tai-nghe-ep-oppo-mh135",  # jack
    "https://www.thegioididong.com/man-hinh-may-tinh/samsung-s3-s30gd-ls27d300gaexxv-27-inch-fhd-ips-100hz-5ms",  # screen
    "https://www.thegioididong.com/may-tinh-bang/xiaomi-redmi-pad-se-8gb?utm_flashsale=1",  # tablet
    "https://www.thegioididong.com/dong-ho-thong-minh/xiaomi-watch-s-3?utm_flashsale=1",  # watch
    "https://www.thegioididong.com/dong-ho-thong-minh/mi-band-9-active?utm_flashsale=1",  # band
]
plist = []

d = await Crawler.async_inspect(
    urls[5], xpath="//*[@id='productld']/text()", encoding="utf-8"
)


d = json.loads(d[0])
d

{'@context': 'https://schema.org',
 '@type': 'Product',
 'name': 'Tai nghe Có dây OPPO MH135',
 'url': 'https://www.thegioididong.com/tai-nghe/tai-nghe-ep-oppo-mh135',
 'image': {'@type': 'ImageObject',
  'contentUrl': 'https://cdn.tgdd.vn/Products/Images/54/327689/tai-nghe-ep-oppo-mh135-090724-040837-600x600.jpg'},
 'description': 'Tai nghe Có dây OPPO MH135 chính hãng, âm thanh chất lượng cao. Giao hàng tận nơi.',
 'sku': '327689',
 'mpn': '327689',
 'brand': {'@type': 'Brand', 'name': ['OPPO']},
 'review': [{'@type': 'Review',
   'author': {'@type': 'Person', 'name': 'Đỗ Xuân Minh'},
   'datePublished': '1/20/2025 1:34:53 PM: dd/MM/yyyy',
   'reviewBody': 'Chất liệu nhựa bọc dây không bền, nghe hay',
   'description': None,
   'reviewRating': {'@type': 'Rating', 'bestRating': 5, 'ratingValue': 4.0}},
  {'@type': 'Review',
   'author': {'@type': 'Person', 'name': 'Đ V Hít'},
   'datePublished': '8/11/2024 10:41:57 PM: dd/MM/yyyy',
   'reviewBody': 'Âm thanh rất ok , rõ sắc nét, âm tr

In [40]:
def parse_common_info(data: dict):
    prd = Product(
        product_id=data["sku"].strip(),
        name=data["name"].strip(),
        price=int(data["offers"]["price"]),
        brand=data["brand"]["name"][0].strip(),
        url=data["url"].strip(),
    )

    if data["aggregateRating"]:
        prd.rating = data["aggregateRating"]["ratingValue"]
        prd.reviews_count = int(data["aggregateRating"]["reviewcount"])

    released_value = [
        i["value"].strip()
        for i in data["additionalProperty"]
        if i["name"] == "Thời điểm ra mắt"
        or i["name"] == "Thời gian ra mắt"
        or i["name"] == "Năm ra mắt"
    ]
    weight_value = [
        i["value"].split(" - ")[-1].removesuffix("g").removeprefix("Nặng").strip()
        for i in data["additionalProperty"]
        if i["name"] == "Kích thước, khối lượng" or i["name"] == "Khối lượng"
    ]
    jack_value = [
        i["value"].strip()
        for i in data["additionalProperty"]
        if i["name"] == "Jack cắm"
    ]

    prd.release_date = released_value[0] if released_value else None

    # check device type
    if prd.url.split("/")[3] == "dtdd":
        prd.category = (
            "Smartphone" if weight_value and float(weight_value[0]) > 135.0 else "Phone"
        )
    elif prd.url.split("/")[3] == "laptop":
        prd.category = "Laptop"
    elif prd.url.split("/")[3] == "dong-ho-thong-minh":
        prd.category = (
            "Smartwatch"
            if weight_value and float(weight_value[0]) > 30.0
            else "Smartband"
        )
    elif prd.url.split("/")[3] == "may-tinh-bang":
        prd.category = "Tablet"
    elif prd.url.split("/")[3] == "tai-nghe":
        if weight_value and float(weight_value[0]) > 190.0:
            prd.category = "Headphone"
        elif jack_value:
            prd.category = "Earphone"
        elif not jack_value:
            prd.category = "Earbuds"
    else:
        prd.category = "Screen"

    return {
        "product_id": prd.product_id,
        "name": prd.name,
        "price": prd.price,
        "brand": prd.brand,
        "category": prd.category,
        "url": prd.url,
        "rating": prd.rating,
        "reviews_count": prd.reviews_count,
        "release_date": prd.release_date,
    }


for i in urls:
    product = None
    data = await Crawler.async_inspect(
        i, xpath="//*[@id='productld']/text()", encoding="utf-8"
    )

    data = json.loads(data[0])

    # check device type
    if i.split("/")[3] == "dtdd":
        product = Smartphone(**parse_common_info(data))
    elif i.split("/")[3] == "latop":
        product = Laptop(**parse_common_info(data))
    elif i.split("/")[3] == "may-tinh-bang":
        product = Tablet(**parse_common_info(data))
    elif i.split("/")[3] == "dong-ho-thong-minh":
        product = Smartwatch(**parse_common_info(data))
    elif i.split("/")[3] == "tai-nghe":
        product = Earphone(**parse_common_info(data))
    else:
        product = Screen(**parse_common_info(data))

    # parse common info

    plist.append(product.info())


plist

[{'id': '333347',
  'name': 'Điện thoại Samsung Galaxy S25 Ultra 5G 12GB/256GB',
  'price': 26880000,
  'brand': 'Samsung',
  'category': 'Smartphone',
  'chipset': None,
  'ram': None,
  'storage': None,
  'screen_size': None,
  'screen_tech': None,
  'screen_resolution': None,
  'screen_frequency': None,
  'operating_system': None,
  'battery': None,
  'rating': 4.9,
  'reviews_count': 13380,
  'url': 'https://www.thegioididong.com/dtdd/samsung-galaxy-s25-ultra',
  'created_at': '01/2025',
  'updated_at': '2025-08-31 13:50:27'},
 {'id': '207956',
  'name': 'Điện thoại Nokia 220 4G',
  'price': 980000,
  'brand': 'Nokia',
  'category': 'Phone',
  'chipset': None,
  'ram': None,
  'storage': None,
  'screen_size': None,
  'screen_tech': None,
  'screen_resolution': None,
  'screen_frequency': None,
  'operating_system': None,
  'battery': None,
  'rating': 4.9,
  'reviews_count': 61302,
  'url': 'https://www.thegioididong.com/dtdd/nokia-220-4g',
  'created_at': '06/2024',
  'updated_at